In [ ]:
%load_ext autoreload
%autoreload 2

## Imports

In [ ]:
from dmsbatch import create_batch_client, create_blob_client
import datetime

## First create a batch client from the config file

In [ ]:
client = create_batch_client('../tests/data/dmsbatch.config')
blob_client = create_blob_client('../tests/data/dmsbatch.config')

## Application packages
To copy large files and programs it is best to zip (or targz) them and upload them as application packages

Application packages are setup separately in either azure management apis or from the web console or cli tool

These are referenced here by their name and version
e.g. DSM2, python and other programs

In [ ]:
app_pkgs = [('dsm2', '8.2.c5aacef7', 'DSM2-8.2.c5aacef7-win32/bin'),
            ('vista', '1.0-v2019-05-28', 'bin'),
            ('unzip', '5.51-1', 'bin')]

### Show vms available

https://docs.microsoft.com/en-us/azure/virtual-machines/fsv2-series

In [ ]:
#display(client.skus_available())

### Create or resize existing pool
If the pool doesn't exist it will create it
If the pool exists, it will resize to the second arg

In [ ]:
client.create_pool('dsm2winpool',
                   1,
                   app_packages=[(app,version) for app,version,_ in app_pkgs], 
                   vm_size='standard_f2s_v2', 
                   tasks_per_vm=2,
                   os_image_data=('microsoftwindowsserver', 'windowsserver', '2019-datacenter-core')
                  )

### Create job on pool or fail if it exists
Jobs are containers of tasks (things that run on nodes (machines) in the pool). If this exists, the next line will fail

In [ ]:
client.create_job('dsm2jobs','dsm2winpool')

### Create a task
This uses the application package as pre -set up. If not, create one https://docs.microsoft.com/en-us/azure/batch/batch-application-packages

In [ ]:
tsnow = str(datetime.datetime.now().timestamp()).split('.')[0]
task_name = f'hydro_version_{tsnow}'
cmd_string = client.wrap_cmd_with_app_path('hydro -v & unzip -v',app_pkgs)
print(task_name)
print(cmd_string)

In [ ]:
blob_client.blob_client.account_name

In [ ]:
import dmsbatch
permissions = dmsbatch.commands.azureblob.BlobPermissions.WRITE
# |helpers.azureblob.BlobPermissions.ADD|helpers.azureblob.BlobPermissions.CREATE
output_dir_sas_url = blob_client.get_container_sas_url(
                                                   'dsm2jobs',
                                                   permissions)
print(output_dir_sas_url)

In [ ]:
std_out_files = client.create_output_file_spec(
    '../std*.txt', output_dir_sas_url, blob_path=f'{task_name}')

In [ ]:
hydro_v_task = client.create_task(task_name,cmd_string,output_files=[std_out_files])

### Next submit the task and wait 

In [ ]:
client.submit_tasks('dsm2jobs',[hydro_v_task])

## Finally resize the pool to 0 to save costs

In [ ]:
client.resize_pool('dsm2winpool',0)